# Notebook 02 — Cleaning (CLEAN Layer)
---
## Goal
Transform RAW ingestion output into a clean, standardized, analytics-ready layer.
No business logic or aggregations here — only normalization, deduplication, and type casting.

## Inputs
- `data/raw/discover_snapshot.parquet`
- `data/raw/titles_metadata_raw.parquet`
- `data/raw/providers_lookup.parquet`

## Outputs
- `data/clean/titles_clean.parquet`
- `data/clean/provider_availability_clean.parquet`
- `data/clean/manifest.json`

## Cleaning Rules
- `release_year`: cast to nullable Int64 — anomalies outside 1900–2026 set to null
- `recency_years`: `2026 - release_year` — null if release_year is null
- `vote_weight`: `log(1 + vote_count)` — dampens blockbuster dominance in weighted metrics
- `runtime_min`: kept null if missing, never imputed
- `primary_genre`, `origin_country`: kept null if missing, never imputed
- Empty strings replaced with null across all string columns
- Dedup titles key: `(tmdb_id, media_type)`
- Dedup availability key: `(tmdb_id, media_type, provider_id, country)`
- Paramount Plus: two provider IDs (2303, 2616) unified via canonical name join
---

## Imports & Paths

In [1]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path

# Project root detection — works whether running from /notebooks or project root
NOTEBOOK_DIR = Path(__file__).parent if "__file__" in dir() else Path.cwd()

PROJECT_ROOT = NOTEBOOK_DIR
for candidate in [NOTEBOOK_DIR, NOTEBOOK_DIR.parent, NOTEBOOK_DIR.parent.parent]:
    if (candidate / "data").exists() and (candidate / "notebooks").exists():
        PROJECT_ROOT = candidate
        break

DATA_RAW_DIR   = str(PROJECT_ROOT / "data" / "raw")
DATA_CLEAN_DIR = str(PROJECT_ROOT / "data" / "clean")
os.makedirs(DATA_CLEAN_DIR, exist_ok=True)

SNAPSHOT_DATE = "2026-02-21"
CURRENT_YEAR  = 2026

print(f"Project root: {PROJECT_ROOT}")
print(f"RAW dir     : {DATA_RAW_DIR}")
print(f"CLEAN dir   : {DATA_CLEAN_DIR}")

Project root: c:\Users\matt\OneDrive\Desktop\Data_Projects\Streaming_Benchmark_Project
RAW dir     : c:\Users\matt\OneDrive\Desktop\Data_Projects\Streaming_Benchmark_Project\data\raw
CLEAN dir   : c:\Users\matt\OneDrive\Desktop\Data_Projects\Streaming_Benchmark_Project\data\clean


## Load RAW Files
No transformations here — pure load to preserve the original RAW state.

In [2]:
discover_raw  = pd.read_parquet(os.path.join(DATA_RAW_DIR, "discover_snapshot.parquet"))
metadata_raw  = pd.read_parquet(os.path.join(DATA_RAW_DIR, "titles_metadata_raw.parquet"))
providers_raw = pd.read_parquet(os.path.join(DATA_RAW_DIR, "providers_lookup.parquet"))

print(f"discover_raw  : {discover_raw.shape}")
print(f"metadata_raw  : {metadata_raw.shape}")
print(f"providers_raw : {providers_raw.shape}")
print()
print(providers_raw)

discover_raw  : (127399, 5)
metadata_raw  : (46973, 14)
providers_raw : (6, 3)

   provider_id             provider_name  canonical_provider
0            9        Amazon Prime Video  Amazon Prime Video
1          350                  Apple TV       Apple TV Plus
2          337               Disney Plus         Disney Plus
3            8                   Netflix             Netflix
4         2616  Paramount Plus Essential      Paramount Plus
5         2303    Paramount Plus Premium      Paramount Plus


## Clean Titles Metadata
Produces `titles_clean` — one row per unique `(tmdb_id, media_type)` with standardized types, recency fields, and vote weight.

In [3]:
titles = metadata_raw.copy()

# Dedup — same title can appear multiple times in RAW
before = len(titles)
titles = titles.drop_duplicates(subset=["tmdb_id", "media_type"], keep="first").reset_index(drop=True)
print(f"Dedup titles: {before} -> {len(titles)} rows")

# release_year — cast to nullable Int64, nullify anomalies outside 1900-2026
titles["release_year"] = pd.to_numeric(titles["release_year"], errors="coerce").astype("Int64")
anomaly_mask = titles["release_year"].notna() & (
    (titles["release_year"] < 1900) | (titles["release_year"] > CURRENT_YEAR)
)
print(f"Anomalous release_year (set to null): {anomaly_mask.sum()}")
titles.loc[anomaly_mask, "release_year"] = pd.NA

# vote_count -> Int64, runtime_min -> float, nulls kept as-is (never imputed)
titles["vote_count"]  = pd.to_numeric(titles["vote_count"],  errors="coerce").astype("Int64")
titles["runtime_min"] = pd.to_numeric(titles["runtime_min"], errors="coerce")

# Strip whitespace and treat empty strings as null
str_cols = ["title", "primary_genre", "genres", "origin_country", "original_language", "media_type"]
for col in str_cols:
    titles[col] = titles[col].str.strip().replace("", None)

# Recency bucket — dynamic, relative to CURRENT_YEAR
# Recent: >= 2021 | Mid: 2011-2020 | Legacy: <= 2010
def recency_bucket(year):
    if pd.isna(year):
        return None
    y = int(year)
    if y >= CURRENT_YEAR - 5:
        return "Recent"
    elif y > CURRENT_YEAR - 15:
        return "Mid"
    else:
        return "Legacy"

titles["recency_bucket"] = titles["release_year"].apply(recency_bucket)

# recency_years — how old is the title relative to the snapshot year
titles["recency_years"] = CURRENT_YEAR - titles["release_year"]
titles.loc[titles["recency_years"] < 0, "recency_years"] = pd.NA

# vote_weight — log-scaled vote_count, dampens blockbuster dominance in weighted metrics
titles["vote_weight"] = np.log1p(titles["vote_count"].fillna(0))

# Final column selection and order
TITLES_COLS = [
    "tmdb_id", "media_type", "title",
    "release_date", "release_year", "recency_years", "recency_bucket",
    "primary_genre", "genres",
    "origin_country", "original_language",
    "vote_average", "vote_count", "vote_weight", "popularity",
    "runtime_min", "snapshot_date",
]
titles = titles[TITLES_COLS]

print(f"\nFinal titles shape: {titles.shape}")
print("\nMissingness:")
print(titles.isna().mean().sort_values(ascending=False).head(8))
print("\nRecency bucket distribution:")
print(titles["recency_bucket"].value_counts())

Dedup titles: 46973 -> 46973 rows
Anomalous release_year (set to null): 2

Final titles shape: (46973, 17)

Missingness:
runtime_min       0.112597
genres            0.032806
primary_genre     0.032806
origin_country    0.009474
recency_years     0.003811
release_year      0.003811
recency_bucket    0.003811
tmdb_id           0.000000
dtype: float64

Recency bucket distribution:
recency_bucket
Mid       16698
Legacy    15531
Recent    14565
Name: count, dtype: int64


## Clean Provider Availability
Produces `provider_availability_clean` — one row per unique `(tmdb_id, media_type, provider_id, country)` with canonical provider name joined.

In [4]:
availability = discover_raw.copy()

# Dedup — key is (tmdb_id, media_type, provider_id, country)
before = len(availability)
availability = availability.drop_duplicates(
    subset=["tmdb_id", "media_type", "provider_id", "country"]
).reset_index(drop=True)
print(f"Dedup availability: {before} -> {len(availability)} rows")

# Join canonical_provider — Paramount Plus (2303, 2616) unified under one canonical name
availability = availability.merge(
    providers_raw[["provider_id", "canonical_provider"]],
    on="provider_id",
    how="left"
)

unmatched = availability["canonical_provider"].isna().sum()
print(f"Unmatched provider_ids: {unmatched}")

AVAIL_COLS = [
    "tmdb_id", "media_type", "provider_id", "canonical_provider",
    "country", "snapshot_date"
]
availability = availability[AVAIL_COLS]

print(f"\nFinal availability shape: {availability.shape}")
print("\nTitles per provider:")
print(availability.groupby("canonical_provider")["tmdb_id"].nunique().sort_values(ascending=False))
print("\nTitles per country:")
print(availability.groupby("country")["tmdb_id"].nunique().sort_values(ascending=False))

Dedup availability: 127399 -> 127399 rows
Unmatched provider_ids: 0

Final availability shape: (127399, 6)

Titles per provider:
canonical_provider
Amazon Prime Video    26477
Netflix               14459
Disney Plus            5834
Paramount Plus         2563
Apple TV Plus           326
Name: tmdb_id, dtype: int64

Titles per country:
country
GB    26479
US    26117
DE    19828
FR    12881
ES    12756
IT    12420
KR    11309
Name: tmdb_id, dtype: int64


## Save CLEAN Files + Manifest

In [5]:
titles_path = os.path.join(DATA_CLEAN_DIR, "titles_clean.parquet")
avail_path  = os.path.join(DATA_CLEAN_DIR, "provider_availability_clean.parquet")

titles.to_parquet(titles_path, index=False)
availability.to_parquet(avail_path, index=False)

print(f"Saved: {titles_path}")
print(f"Saved: {avail_path}")

manifest = {
    "snapshot_date": SNAPSHOT_DATE,
    "current_year": CURRENT_YEAR,
    "source": "TMDB",
    "layer": "clean",
    "row_counts": {
        "titles_clean": int(len(titles)),
        "provider_availability_clean": int(len(availability)),
    },
    "missingness": {
        col: round(float(titles[col].isna().mean()), 4)
        for col in titles.columns
        if titles[col].isna().any()
    },
    "recency_buckets": titles["recency_bucket"].value_counts().to_dict(),
    "providers": sorted(availability["canonical_provider"].unique().tolist()),
    "countries": sorted(availability["country"].unique().tolist()),
    "cleaning_rules": {
        "release_year_anomalies": "set to null if < 1900 or > 2026",
        "runtime_min": "kept null if missing, never imputed",
        "empty_strings": "replaced with null",
        "dedup_titles": "(tmdb_id, media_type)",
        "dedup_availability": "(tmdb_id, media_type, provider_id, country)",
    }
}

manifest_path = os.path.join(DATA_CLEAN_DIR, "manifest.json")
with open(manifest_path, "w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2)

print(f"\nManifest written to: {manifest_path}")

Saved: c:\Users\matt\OneDrive\Desktop\Data_Projects\Streaming_Benchmark_Project\data\clean\titles_clean.parquet
Saved: c:\Users\matt\OneDrive\Desktop\Data_Projects\Streaming_Benchmark_Project\data\clean\provider_availability_clean.parquet

Manifest written to: c:\Users\matt\OneDrive\Desktop\Data_Projects\Streaming_Benchmark_Project\data\clean\manifest.json


## Sanity Checks
Final validation before passing the CLEAN layer to Notebook 03.

In [7]:
print("=== TITLES CLEAN ===")
print(f"Shape: {titles.shape}")
print(f"\nmedia_type distribution:\n{titles['media_type'].value_counts()}")
print(f"\nrecency_bucket distribution:\n{titles['recency_bucket'].value_counts()}")
print(f"\nTop 10 primary_genre:\n{titles['primary_genre'].value_counts().head(10)}")

print("\n=== AVAILABILITY CLEAN ===")
print(f"Shape: {availability.shape}")
print(f"\nTitles per provider x country:")
pivot = (
    availability.groupby(["canonical_provider", "country"])["tmdb_id"]
    .nunique()
    .unstack(fill_value=0)
)
print(pivot.to_string())

print("\n=== MISSINGNESS TITLES ===")
missing = titles.isna().mean().sort_values(ascending=False)
missing = missing[missing > 0]
print(missing)

=== TITLES CLEAN ===
Shape: (46973, 17)

media_type distribution:
media_type
movie    33657
tv       13316
Name: count, dtype: int64

recency_bucket distribution:
recency_bucket
Mid       16698
Legacy    15531
Recent    14565
Name: count, dtype: int64

Top 10 primary_genre:
primary_genre
Drama          10079
Comedy          7438
Documentary     5860
Action          3435
Animation       2854
Horror          2696
Crime           1919
Thriller        1844
Reality         1256
Romance         1202
Name: count, dtype: int64

=== AVAILABILITY CLEAN ===
Shape: (127399, 6)

Titles per provider x country:
country               DE    ES    FR     GB    IT    KR     US
canonical_provider                                            
Amazon Prime Video  8010     0     0  13387     0     0  14398
Apple TV Plus        309   309   305    311   314   305    317
Disney Plus         3482  3519  3612   4058  3566  2663   2346
Netflix             8693  9385  8644   8853  9004  8480   7788
Paramount Plus    